<a href="https://colab.research.google.com/github/shrkamat/cloud/blob/master/ai_stack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install git+https://github.com/m-bain/whisperx.git yt-dlp

In [ ]:
# ! wget "https://chtbl.com/track/5899E/podtrac.com/pts/redirect.mp3/traffic.omny.fm/d/clips/e73c998e-6e60-432f-8610-ae210140c5b1/69f89044-ce7b-4a37-b8fe-ae2c007b3cf5/61a04639-7c04-4e31-b0b4-b154010cea16/audio.mp3?utm_source=Podcast&in_playlist=4331cc95-c2e7-4689-a79c-ae2c007b3cff" -O audio.mp3
! yt-dlp --format 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4' https://www.youtube.com/watch?v=g_1oiJqE3OI -o "video.mp4" --write-sub
! ffmpeg -i video.mp4 -vn -c:a libmp3lame audio.mp3
! du -sh video.mp4
! du -sh audio.mp3

In [ ]:
# (device, compute) = ('cuda', 'float16')
(device, compute) = ('cpu', 'float32')
import whisperx

wx_model = whisperx.load_model("medium.en", device=device, compute_type=compute)

In [ ]:
audio = whisperx.load_audio("audio.mp3")
result = wx_model.transcribe(audio, batch_size=16)

In [ ]:
import json

# delete words key from result dict
for seg in result_a['segments']:
  del seg['words']

with open("audio_orig.json", "w") as f:
    f.write(json.dumps(result['segments'], indent=4))

In [ ]:
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result_a = whisperx.align(result["segments"], model_a, metadata, audio, device=device, return_char_alignments=False)

In [ ]:
import json
with open("audio_tokenized.json", "w") as f:
    f.write(json.dumps(result_a['segments'], indent=4))

In [ ]:
import math

def seconds_to_timestamp(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    remaining_seconds = seconds % 60
    (ms, sec) = math.modf(remaining_seconds)
    return '{:02d}:{:02d}:{:02d},{:03d}'.format(int(hours), int(minutes), int(sec), int(ms*1000))

with open("audio.srt", "w") as f:
  for idx, seg in enumerate(result_a['segments']):
    res = '\n{:d}\n{:s} --> {:s}\n{:s}\n'.format(int(idx+1), seconds_to_timestamp(seg['start']), seconds_to_timestamp(seg['end']), seg['text'].lstrip())
    f.write(res)